In [1]:
library('fastDummies')
library('pmml')
library('missForest')

Warning message:
"package 'fastDummies' was built under R version 4.0.2"
Loading required package: XML

Warning message:
"package 'missForest' was built under R version 4.0.2"
Loading required package: randomForest

Warning message:
"package 'randomForest' was built under R version 4.0.2"
randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.

Loading required package: foreach

Warning message:
"package 'foreach' was built under R version 4.0.2"
Loading required package: itertools

Warning message:
"package 'itertools' was built under R version 4.0.2"
Loading required package: iterators

Warning message:
"package 'iterators' was built under R version 4.0.2"


In [2]:
data <- read.csv('./hmeq.csv', header=T, na.string=c(""))

In [3]:
ncol(data)

[1] 13

In [4]:
head(data)

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<int>,<int>,<dbl>,<int>,<int>,<dbl>
1,1,1100,25860,39025,HomeImp,Other,10.5,0,0,94.36667,1,9,NA
2,1,1300,70053,68400,HomeImp,Other,7.0,0,2,121.83333,0,14,NA
3,1,1500,13500,16700,HomeImp,Other,4.0,0,0,149.46667,1,10,NA
4,1,1500,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,0,1700,97800,112000,HomeImp,Office,3.0,0,0,93.33333,0,14,NA
6,1,1700,30548,40320,HomeImp,Other,9.0,0,0,101.46600,1,8,37.11361


In [5]:
data <- data.frame( CLAGE = as.numeric(data$CLAGE),
                 CLNO = as.numeric(data$CLNO),
                 DEBTINC = as.numeric(data$DEBTINC),
                 DELINQ = as.numeric(data$DELINQ),
                 DEROG = as.numeric(data$DEROG),
                 LOAN = as.numeric(data$LOAN),
                 MORTDUE = as.numeric(data$MORTDUE),
                 NINQ = as.numeric(data$NINQ),
                 VALUE = as.numeric(data$VALUE),
                 YOJ = as.numeric(data$YOJ),
                 JOB = as.factor(data$JOB),
                 REASON = as.factor(data$REASON),
                 BAD = as.factor(data$BAD),
                 stringsAsFactors = FALSE)

In [6]:
imp <- missForest(data, maxiter = 2, ntree = 10)

  missForest iteration 1 in progress...done!
  missForest iteration 2 in progress...done!


In [7]:
imp$OOBerror

NRMSE       PFC 
0.1846278 0.1069916

In [8]:
imp_f <- imp$ximp

In [9]:
data_dummies <- dummy_cols(imp_f)
head(data_dummies)

,CLAGE,CLNO,DEBTINC,DELINQ,DEROG,LOAN,MORTDUE,NINQ,VALUE,YOJ,...,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,REASON_DebtCon,REASON_HomeImp,BAD_0,BAD_1
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,94.36667,9.0,35.89829,0,0,1100,25860.0,1.0,39025.0,10.5,...,0,0,1,0,0,0,0,1,0,1
2,121.83333,14.0,35.67594,2,0,1300,70053.0,0.0,68400.0,7.0,...,0,0,1,0,0,0,0,1,0,1
3,149.46667,10.0,24.76203,0,0,1500,13500.0,1.0,16700.0,4.0,...,0,0,1,0,0,0,0,1,0,1
4,144.40410,25.6,39.25600,2,1,1500,42126.3,1.2,76258.5,12.6,...,0,0,1,0,0,0,1,0,0,1
5,93.33333,14.0,30.78656,0,0,1700,97800.0,0.0,112000.0,3.0,...,0,1,0,0,0,0,0,1,1,0
6,101.46600,8.0,37.11361,0,0,1700,30548.0,1.0,40320.0,9.0,...,0,0,1,0,0,0,0,1,0,1


In [10]:
abt <- within(data_dummies, rm("JOB", "REASON", "BAD_1", "BAD_0"))
abt$BAD =data_dummies$BAD

In [11]:
head(abt)

,CLAGE,CLNO,DEBTINC,DELINQ,DEROG,LOAN,MORTDUE,NINQ,VALUE,YOJ,BAD,JOB_Mgr,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,REASON_DebtCon,REASON_HomeImp
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,94.36667,9.0,35.89829,0,0,1100,25860.0,1.0,39025.0,10.5,1,0,0,1,0,0,0,0,1
2,121.83333,14.0,35.67594,2,0,1300,70053.0,0.0,68400.0,7.0,1,0,0,1,0,0,0,0,1
3,149.46667,10.0,24.76203,0,0,1500,13500.0,1.0,16700.0,4.0,1,0,0,1,0,0,0,0,1
4,144.40410,25.6,39.25600,2,1,1500,42126.3,1.2,76258.5,12.6,1,0,0,1,0,0,0,1,0
5,93.33333,14.0,30.78656,0,0,1700,97800.0,0.0,112000.0,3.0,0,0,1,0,0,0,0,0,1
6,101.46600,8.0,37.11361,0,0,1700,30548.0,1.0,40320.0,9.0,1,0,0,1,0,0,0,0,1


In [ ]:
dm_modelformula

In [12]:
model <- glm(formula = BAD ~ ., family = binomial(link = "logit"), data = abt)

s <- summary(model)

In [13]:
s


Call:
glm(formula = BAD ~ ., family = binomial(link = "logit"), data = abt)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2804  -0.5863  -0.4011  -0.2114   3.4876  

Coefficients: (2 not defined because of singularities)
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)    -3.626e+00  3.221e-01 -11.256  < 2e-16 ***
CLAGE          -4.161e-03  5.373e-04  -7.744 9.61e-15 ***
CLNO           -2.571e-02  4.540e-03  -5.665 1.47e-08 ***
DEBTINC         1.054e-01  6.512e-03  16.185  < 2e-16 ***
DELINQ          7.814e-01  3.959e-02  19.736  < 2e-16 ***
DEROG           5.378e-01  4.882e-02  11.016  < 2e-16 ***
LOAN           -1.952e-05  4.292e-06  -4.547 5.43e-06 ***
MORTDUE        -7.139e-06  1.803e-06  -3.960 7.51e-05 ***
NINQ            1.369e-01  2.094e-02   6.535 6.36e-11 ***
VALUE           4.758e-06  1.288e-06   3.693 0.000222 ***
YOJ            -1.037e-02  5.868e-03  -1.766 0.077314 .  
JOB_Mgr        -5.432e-01  2.307e-01  -2.355 0.018517 *  

In [ ]:
print(s)

In [14]:
saveXML(pmml(model, model.name="General_Regression_Model",
             app.name="Rattle/PMML",
             description="Generalized Linear Regression Model"), "R_HMEQ.xml")

[1] "R_HMEQ.xml"

In [ ]:
write.csv(abt, './hmeq_dummies.csv', row.names=FALSE)